In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

c:\Users\paulo\miniconda3\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '555125a533659be853c7c9108b96b123', 'Date': 'Tue, 27 Aug 2024 04:10:26 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [5]:
import json
data = json.load(open("reviews.json"))
data['reviews']


[{'review': 'Excellent teacher, clear explanations and very helpful.',
  'professor': 'Dr. John Smith',
  'subject': 'Computer Science',
  'stars': 4.5},
 {'review': 'Knowledgeable but tough exams. Attend all lectures.',
  'professor': 'Dr. Emily Johnson',
  'subject': 'Biology',
  'stars': 3.8},
 {'review': 'Passionate and engaging. Be prepared for heavy reading.',
  'professor': 'Prof. Michael Lee',
  'subject': 'History',
  'stars': 4.2},
 {'review': 'Makes complex concepts easy to understand. Great problem-solving sessions.',
  'professor': 'Dr. Sarah Chen',
  'subject': 'Physics',
  'stars': 4.7},
 {'review': 'Very knowledgeable but lectures can be dry. Good discussions.',
  'professor': 'Prof. Robert Taylor',
  'subject': 'English Literature',
  'stars': 3.5},
 {'review': 'Engaging and relatable. Uses great real-world examples.',
  'professor': 'Dr. Lisa Wong',
  'subject': 'Psychology',
  'stars': 4.8},
 {'review': 'Thorough explanations but fast-paced. Practice regularly.',
  '

In [8]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [9]:
processed_data[0]

{'values': [-0.034996036,
  -0.016054751,
  -0.02766475,
  -0.009304604,
  -0.023488216,
  -0.018992376,
  -0.04470297,
  0.043604553,
  -0.036222175,
  0.0316497,
  0.04470297,
  0.00012712406,
  -0.040309306,
  -0.06396356,
  0.019784257,
  0.013896237,
  0.01683386,
  0.03614554,
  0.010818118,
  0.0694301,
  0.016476236,
  -0.0003506389,
  0.009694158,
  0.039261978,
  -0.022811286,
  -0.079188116,
  0.013359801,
  -0.00765698,
  -0.010058167,
  -0.016884949,
  0.04999069,
  -0.022083266,
  0.00076354266,
  -0.042965937,
  -0.02917188,
  -0.011737722,
  0.0032457549,
  0.00012742341,
  -0.0059806183,
  -0.01677,
  0.009502574,
  0.025314651,
  -0.005179158,
  0.007248267,
  0.045954652,
  0.009579208,
  -0.040641382,
  0.002338923,
  0.0062200986,
  0.026949503,
  -0.0005021101,
  0.03586455,
  0.05037386,
  0.026949503,
  -0.03841901,
  0.008998069,
  -0.001337896,
  0.045929108,
  0.00016593981,
  -0.030040395,
  0.06580277,
  -0.0067565343,
  0.03338673,
  0.02182782,
  0.026770

In [11]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [12]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}